https://mlflow.org/docs/latest/getting-started/quickstart-2/index.html

In [12]:
!pip install -q tensorflow hyperopt

In [5]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [6]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)
signature = infer_signature(train_x, train_y)

In [7]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [8]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [9]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [10]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2024/07/15 03:46:56 INFO mlflow.tracking.fluent: Experiment with name '/wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 16s 365ms/step - loss: 31.3983 - root_mean_squared_error: 5.6034
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 33.2076 - root_mean_squared_error: 5.7625 - val_loss: 32.9036 - val_root_mean_squared_error: 5.7362

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 34.3817 - root_mean_squared_error: 5.8636
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 32.7759 - root_mean_squared_error: 5.7248 - val_loss: 32.0463 - val_root_mean_squared_error: 5.6609

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 32.0718 - root_mean_squared_error: 5.6632
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 31.6130 - root_mean_squared_error: 5.6225 - val_loss: 31.2131 - val_root_mean_squared_error: 5.5869

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 31.5689 - root_mean_squared_error: 5.6186
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s

/opt/conda/lib/python3.11/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 13s 294ms/step - loss: 37.2246 - root_mean_squared_error: 6.1012
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 10.3171 - root_mean_squared_error: 3.0742 - val_loss: 0.8814 - val_root_mean_squared_error: 0.9388

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.9613 - root_mean_squared_error: 0.9804
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8199 - root_mean_squared_error: 0.9032 - val_loss: 0.5274 - val_root_mean_squared_error: 0.7262

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3873 - root_mean_squared_error: 0.6223
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5487 - root_mean_squared_error: 0.7404 - val_loss: 0.5227 - val_root_mean_squared_error: 0.7230

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 

In [16]:
model = mlflow.pyfunc.load_model("models:/wine-quality/1")
type(model)

mlflow.pyfunc.PyFuncModel

In [17]:
model.serve(port=9999)

AttributeError: 'PyFuncModel' object has no attribute 'serve'